In [23]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [49]:
# Read data from excel file

xlsx = r"C:\Users\mfaraday01\Documents\Code\Aging Analysis\credit_debit_records.xlsx"
debit = pd.read_excel (xlsx, 
                     sheet_name="Debits"
                     )
credit = pd.read_excel (xlsx, 
                     sheet_name="Credit"
                     )

In [50]:
# Unpivot "debit df"

debit_unpiv = debit.melt(id_vars=["Zone", "Region", "District"], var_name="Date", value_name="Balance")

In [51]:
# Change datatypes

debit_unpiv[["Zone", "Region", "District"]] = debit_unpiv[["Zone", "Region", "District"]].astype(str)
debit_unpiv["Date"] = pd.to_datetime(debit_unpiv["Date"]).dt.date

In [52]:
# Add new calculated columns

debit_unpiv["Due date"] = debit_unpiv["Date"] + timedelta(days=7)
debit_unpiv["Overdue"] = (datetime.date(datetime.now()) - debit_unpiv["Due date"]).dt.days.astype('int16')

In [53]:
# Categorize overdue time to ages

conditions = [
               (debit_unpiv["Overdue"] <= 7),
               (debit_unpiv["Overdue"] > 7) & (debit_unpiv["Overdue"] <= 14),
               (debit_unpiv["Overdue"] > 14) & (debit_unpiv["Overdue"] <= 21),
               (debit_unpiv["Overdue"] > 21) & (debit_unpiv["Overdue"] <= 60),
               (debit_unpiv["Overdue"] > 60) & (debit_unpiv["Overdue"] <= 120),
               (debit_unpiv["Overdue"] > 120) & (debit_unpiv["Overdue"] <= 180),
               (debit_unpiv["Overdue"] > 180)               
               ]

choices = ["1-7 days", "8-15", "16-21", "22-60 days", "61-120 days", "121-180 days", "Above 180"]

debit_unpiv["Aging Status"] = np.select(conditions, choices)

In [ ]:
# Merge the two df

debit_credit = debit_unpiv.merge(credit, on="District")

In [ ]:
#Sort and re-index df

debit_credit = debit_credit.sort_values(by=["District", "Date"], ascending=True)
debit_credit = debit_credit.reset_index(drop=True)

In [83]:
#Calculate for FIFO balances

FIFO_balance = []
bal_bf = 0
indexor = -1
indexor2 = 0
dc_columns = list(debit_credit["District"].unique())

for i in debit_credit["Balance"]:
   indexor += 1
   if debit_credit["District"][indexor] == dc_columns[indexor2]:
      bal = min(i, debit_credit["Total Credit"][indexor] - bal_bf)
      FIFO_balance.append(bal)
      bal_bf += bal
   else:
      bal_bf = 0
      indexor2 += 1
      bal = min(i, debit_credit["Total Credit"][indexor] - bal_bf)
      FIFO_balance.append(bal)
      bal_bf += bal
      
debit_credit["FIFO balance"] = FIFO_balance

,Zone,Region,District,Date,Balance,Due date,Overdue,Aging Status,Total Credit,FIFO balance
0,2,5,Abenase - UCGL,2021-10-01,0,2021-10-08,227,Above 180,18455,0
1,2,5,Abenase - UCGL,2021-10-08,0,2021-10-15,220,Above 180,18455,0
2,2,5,Abenase - UCGL,2021-10-15,800,2021-10-22,213,Above 180,18455,800
3,2,5,Abenase - UCGL,2021-10-22,1605,2021-10-29,206,Above 180,18455,1605
4,2,5,Abenase - UCGL,2021-10-31,0,2021-11-07,197,Above 180,18455,0
...,...,...,...,...,...,...,...,...,...,...
3483,1,1,Wassa Akropong B,2022-04-08,0,2022-04-15,38,22-60 days,4550,0
3484,1,1,Wassa Akropong B,2022-04-15,0,2022-04-22,31,22-60 days,4550,0
3485,1,1,Wassa Akropong B,2022-04-22,0,2022-04-29,24,22-60 days,4550,0
3486,1,1,Wassa Akropong B,2022-04-29,0,2022-05-06,17,16-21,4550,0


In [ ]:
#